#**Retrieval-Augmented Generation (RAG)**
Retrieval-Augmented Generation (RAG) is a technique that enhances the performance of Large Language Models (LLMs) by combining retrieval-based methods with generation-based models. This approach improves the accuracy, relevance, and factual consistency of generated responses.



##**How RAG Works?**
RAG consists of two main components:

Retrieval Component (Retriever)

1. Searches a database (such as a vector database like ChromaDB, Pinecone, or FAISS) to fetch relevant documents based on the user's query.
2. Uses embeddings to perform similarity search over stored knowledge.
3. Ensures that the model has access to up-to-date and relevant information, beyond its pre-trained knowledge.

Generation Component (LLM)

1. The retrieved documents are passed as context to an LLM (e.g., OpenAI GPT, Mistral, Llama 2, or Hugging Face models).
2. The LLM generates a response based on both the retrieved information and its inherent knowledge.

##Steps to perform RAG
1. Load Dataset (Using which we will perform RAG)
2. Read Files
3. Split Text
4. Create Index
5. Load Index
6. Similarity Search and Response Generation



###1. Load our **dataset** from **github**




In [ ]:
!git clone https://github.com/cloudyuga/genai.git


Cloning into 'genai'...
remote: Enumerating objects: 1757, done.
remote: Counting objects: 100% (214/214), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 1757 (delta 79), reused 165 (delta 64), pack-reused 1543 (from 1)
Receiving objects: 100% (1757/1757), 3.70 MiB | 7.63 MiB/s, done.
Resolving deltas: 100% (852/852), done.


In [ ]:
%cd genai
!git checkout hr


/content/genai
Branch 'hr' set up to track remote branch 'hr' from 'origin'.
Switched to a new branch 'hr'


In [ ]:
# List the Markdown files
!ls -R hrdataset/


hrdataset/:
employees  policies

hrdataset/employees:
101_Priya_Sharma.md  104_Karan_Kapoor.md   107_Amit_Verma.md	   110_Aditya_Jain.md
102_Rohit_Mehra.md   105_Sunita_Patil.md   108_Rajesh_Kulkarni.md  payroll_information.md
103_Anjali_Das.md    106_Neha_Malhotra.md  109_Meera_Iyer.md

hrdataset/policies:
employee_benefits.md  holiday_calendar.md  training_and_development.md
events_calendar.md    leave_policies.md


####Install Dependencies

In [ ]:
!pip install langchain_community
!pip install langchain_text_splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.4/412.4 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.31
    Uninstalling langchain-core-0.3.31:
      Successfully uninstalled langchain-core-0.3.31
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.15
    Uninstalling langchain-0.3.15:
      Successfully uninstalled langchain-0.3.15


In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
employee_files_path = "hrdataset/employees"
policy_files_path = "hrdataset/policies"
persist_directory = "hr_vector_index"

###2. Read Our Files and convert into text

In [ ]:
import os
def read_markdown_files(directory):
    """Read and load content from all Markdown files in a directory."""
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith(".md"):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding='utf-8') as f:
                documents.append({"filename": filename, "content": f.read()})
    return documents

In [ ]:
employee_docs = read_markdown_files(employee_files_path)
policy_docs = read_markdown_files(policy_files_path)
all_docs = employee_docs + policy_docs

###3. Split the text into chunks

In [ ]:
def split_text(documents, chunk_size=1000, chunk_overlap=20):
    """Split text documents into manageable chunks."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        is_separator_regex=False
    )
    chunks = []
    for doc in documents:
        # Split the document into chunks
        doc_chunks = text_splitter.create_documents([doc["content"]])
        # Add metadata (e.g., filename) to each chunk
        for chunk in doc_chunks:
            chunk.metadata = {"filename": doc["filename"]}
        chunks.extend(doc_chunks)
    return chunks

In [ ]:
# Split all documents into chunks
chunks = split_text(all_docs)

In [ ]:
print(chunks[5])

page_content='# Employee Profile: Amit Verma

## Basic Information
- **Employee ID:** 107
- **Name:** Amit Verma
- **Role:** CEO
- **Department:** Executive
- **Manager:** -
- **Contact:** +91-1234567890
- **Joining Date:** 2016-02-01
- **Date of Birth:** 1975-06-15
- **Hobbies:** Golf, Reading

## Performance Ratings
- **2019:** 5.0
- **2020:** 5.0
- **2021:** 5.0
- **2022:** 5.0

## Onboarding Status
- N/A' metadata={'filename': '107_Amit_Verma.md'}


In [ ]:
!pip install chromadb

###4. Create Index

In [ ]:
def create_chroma_index(chunks, persist_directory):
    """Create and persist a ChromaDB index."""
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=persist_directory)
    print(f"ChromaDB index created and saved in {persist_directory}.")
    return vectordb

In [ ]:
create_chroma_index(chunks, persist_directory)

ChromaDB index created and saved in hr_vector_index.


###5. Load Index

In [ ]:
def load_chroma_index(persist_directory=persist_directory):
    """Load an existing ChromaDB index."""
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
    print(f"ChromaDB index loaded from {persist_directory}.")
    return vectordb

###6. Similarity Search and Response Generation using LLM

In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI

In [ ]:
from google.colab import userdata
api_key = userdata.get('openai_api_key')

In [ ]:
client = OpenAI(api_key=api_key)

In [ ]:
def generate_response(context, question):
    """Generate a response using OpenAI."""
    try:
        messages = [
            {"role": "system", "content": "You are an assistant that answers questions based on the provided content."},
            {"role": "user", "content": f"Context: {context}\n\nQuestion: {question}\n\nAnswer:"}
        ]
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Replace with preferred model
            messages=messages,
            max_tokens=150,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error generating response: {e}"

In [ ]:
def process_question(question):
    if not question:
        return "Please provide a question."

    # Step 1: load ChromaDB
    vectordb = load_chroma_index(persist_directory=persist_directory)

    # Step 2: Perform similarity search
    try:
        docs = vectordb.similarity_search(question)
        if not docs:
            return "No relevant information found."

        # Step 3: Generate a response using the retrieved context
        context = docs[0].page_content
        response = generate_response(context, question)
        return response
    except Exception as e:
        return f"Error during similarity search or response generation: {str(e)}"

In [ ]:
print("Response:",process_question("Give me the summary of leave policy in 20 words"))

ChromaDB index loaded from hr_vector_index.
Response: Annual leave: 18 days; sick leave: 12 days; maternity: 6 months; paternity: 15 days; compensatory leave for extra work.
